In [4]:
##Module to pull out all of the date placeholder values for Yahoo options
##Test Code used to experiment with html data for later function definitions

#perform HTML request using requests module
import requests

#define stock ticker input value any stock symbol can be used for 'tikr'
tikr = 'MO'
headers = {'User-Agent': 'Chrome/46.0.2490.80', 'Content-Type': 'text/html',}
#get HTML data
r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr, headers, timeout=5)
#determine and output if request was good
q = r.status_code
print(q)
#organize html data using Beautiful Soup
from bs4 import BeautifulSoup
#turn HTML content into string for Python string processing
soup = BeautifulSoup(r.content, 'lxml')
s = str(soup)


#use string serch queries to find desired values
exp = 'expirationDates":['     #beginning of string search
a = s.find(exp)+len(exp)       #set var 'a' to start
#print(s[a:a+200])             #test code
b = s.find(']',a)              #end of string search set var 'b' to end
li = s[a:b]                    #extract desired string 'li' from s

#initilize variables for processing string 'li'
end = ','
s = 0
e = 0
pl = []

#modify list pl to include each value of li separated by a comma 'end'
while s < len(li) and e != -1:
    beg = s
    e = li.find(end, beg)
    pl = pl + [li[beg:e]]
    s = e+1

#account for unique case that the last value does not have a comma at the end
pl[-1]=[pl[-1]+li[-1]]
#check output against starting string
#print(li)  
print(pl)
print(len(pl))






200
1507248000,1507852800,1508457600,1509062400,1509667200,1510272000,1510876800,1511481600,1513296000,1516320000,1521158400,1529020800,1537488000,1547769600
['1507248000', '1507852800', '1508457600', '1509062400', '1509667200', '1510272000', '1510876800', '1511481600', '1513296000', '1516320000', '1521158400', '1529020800', '1537488000', ['1547769600']]
14


In [1]:
##Function to simplify date requests
##Returns a list of strings which can be input into HTML query for different options calls

import requests

def OptionsFun(tikr):

    tikr = str(tikr)

    headers = {'User-Agent': 'Chrome/46.0.2490.80', 'Content-Type': 'text/html',}
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr, headers, timeout=5)
    q = r.status_code

    from bs4 import BeautifulSoup

    soup = BeautifulSoup(r.content, 'lxml')
    s = str(soup)

    #use string serch queries to find desired values
    exp = 'expirationDates":['
    a = s.find(exp)+len(exp)
    #print(a)
    b = s.find(']',a)
    li = s[a:b]
    #print(li)
    end = ','
    #initialize placeholder variables
    s = 0
    e = 0
    pl = []

    while s < len(li) and e != -1:
        beg = s
        e = li.find(end, beg)
        pl = pl + [li[beg:e]]
        s = e+1
    pl[-1]=pl[-1]+li[-1]
    return pl




In [2]:
## Test function output
k = OptionsFun('MO')
print(k)
print(len(k))

['1507248000', '1507852800', '1508457600', '1509062400', '1509667200', '1510272000', '1510876800', '1511481600', '1513296000', '1516320000', '1521158400', '1529020800', '1537488000', '1547769600']
14


In [ ]:
##Function to pull options data from yahoo finance for all listed execution dates using OptionsFun
#Returns csv files labeled by stock symbol and execution data which contain options data including: 
#...strike price, implied volatility, volume, and contract name

import requests
import pandas as pd


tikr = 'MO'
k = OptionsFun(tikr)

for i in range(len(k)):
    
    r = requests.get('https://finance.yahoo.com/quote/'+tikr+'/options?p='+tikr+'&date='+k[i])

    from lxml import html

    tree = html.fromstring(r.content)
    #read date for options execution
    date = tree.xpath('//span[@class="Fz(s) Mend(10px)"]/text()')[0]
    ##Use requests module with html reader to get list of options
    strike = tree.xpath('//a[@href="/quote/'+tikr+'/options?strike=false&straddle=false"]/text()')  #strike price
    IV = tree.xpath('//td[@class="data-col10 Ta(end) Pstart(7px) Pend(6px)"]/text()') #implied volatility
    volume = tree.xpath('//td[@class="data-col8 Ta(end) Pstart(7px)"]/text()') #volume
    contract = tree.xpath('//a[@class="Fz(s) Ell C($c-fuji-blue-1-b)"]/text()') #contract name 'P' indicates put 'C' indicates call
    print(date)
    for i in range(len(contract)):
        strike[i] = float(strike[i])
        volume[i] = int(volume[i])
        IV[i] = float(IV[i].strip('%'))/100

    options = pd.DataFrame({'strike':strike, 'IV':IV, 'volume':volume}, index = contract)
    #print(options)
    #file_name = str(date)
    #options.to_csv(tikr+'_'+date+'2.csv')
